# For Display:
### Run the notebook, then uncomment and run these cells

In [55]:
# checkout_visit

alt.Chart(...)

If run locally with the included small sample of the checkout information there will be a fraction (5/1000) of the total records of checkouts available for comparison.  Please compare this reduced visualization to the comparison made with the full dataset, included as "comapre_visits_checkouts_years_full.html"

In [56]:
# branch_traffic

alt.Chart(...)

In [57]:
# branch_traffic2

alt.Chart(...)

In [58]:
# useage

alt.LayerChart(...)

# The Code to produce these visualizations

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import altair as alt
from altair_saver import save
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
from functools import reduce

In [8]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('LibraryProject') \
    .getOrCreate() 

sc = spark.sparkContext

### Path to the csv file for checkouts: adjust as needed

In [9]:
# Checkout information
df = spark.read.option("header", True).csv('../ProjectData/checkouts.csv')

In [10]:
# if running with the included small sample of the total checkouts dataset
# update with your path...not to insult your intelligence 

# df = spark.read.option("header", True).csv('../ProjectData/smaller_sample_checkouts.csv')

## Configure Date information

In [11]:
df_1 = df.withColumn('Checkout', regexp_extract('CheckoutDateTime', r"([0-9\/]+)", 1))

In [12]:
df_2 = df_1.withColumn('Checkout', to_date(col("Checkout"), 'MM/d/yyyy'))

In [13]:
df_3 = df_2.withColumn('year', year('Checkout'))

In [14]:
df_ = df_3.withColumn('month', month('Checkout'))

## Checkouts by Year

In [15]:
df_.createOrReplaceTempView("df_full")

In [16]:
query = """
SELECT year, count(ID)
FROM df_full
GROUP BY year
ORDER BY year
"""

In [17]:
df_count_years = spark.sql(query)

In [18]:
df_count = df_count_years.toPandas()
df_count = df_count.rename({'count(ID)':'Checkouts'}, axis=1)
# df_count

## Add Door Count Information (Visits to the branches)

In [19]:
url = "https://gist.githubusercontent.com/Cameron-Grams/565526991400ade865e48d1aebb7cdd5/raw/61394eb466550c239a56e7f83eef1a66e4130329/door_counts_all"
df_door = pd.read_csv(url)

In [20]:
df_door['datetime'] = df_door.apply(lambda x: datetime.strptime(x['time_period'], '%Y-%m-%d'), axis = 1)

In [21]:
df_door['year'] = df_door.apply(lambda x: (x['datetime']).year, axis = 1)

## Compare the visits to total checkouts

In [22]:
# monthly total visits
df_month_totals = df_door[df_door['Branch'] == 'Total']

In [23]:
# df_month_totals.head(20)

In [24]:
df_year_totals = df_month_totals.groupby('year')['Total_visits'].sum()

In [25]:
df_yr_visits = pd.DataFrame(df_year_totals)
# df_yr_visits

In [26]:
df_count = df_count.set_index('year')
#df_count

In [27]:
df_compare_totals = df_count.merge(df_yr_visits, on='year', how='outer')

In [28]:
df_compare_totals = df_compare_totals.dropna()
df_compare_totals = df_compare_totals.reset_index()
#df_compare_totals

In [29]:
df_c = pd.melt(df_compare_totals, id_vars=['year'], value_vars = ['Checkouts', 'Total_visits'])

In [30]:
checkout_visit = alt.Chart(df_c).mark_line().encode(
    x = alt.X('year:N'),
    y = alt.Y('value:Q', title='Counts'),
    color = 'variable:N'
).properties(
    width = 500,
    title = "Comparion of Library Visits to Checkout of Physical Items"
)

In [31]:
checkout_visit

alt.Chart(...)

In [32]:
checkout_visit.save('comapre_visits_checkouts_years.html')

# Relative Use by Branch

In [33]:
def door_count_df(df):
    all_branch_list = []

    branches = ['BAL', 'BEA', 'BRO', 'CAP',
        'CEN', 'COL', 'DLR', 'DTH', 'FRE', 'GLK', 'GWD', 'HIP', 'IDC', 'LCY',
        'MAG', 'MGM', 'MON', 'NET', 'NGA', 'NHY', 'QNA', 'RBE', 'SPA', 'SWT', 
        'UNI', 'WAL','WTS']

    for branch in branches:
        df_ = df[ df['Branch']  == branch]
        df_b = df_.groupby('year')['Total_visits'].sum()
#        df_b = df_b.reset_index()
        df_b = pd.DataFrame(df_b)
        df_b = df_b.rename({'Total_visits': branch}, axis=1)
        all_branch_list.append(df_b)

    return_df = reduce(lambda x, y: pd.merge(x, y, left_index=True, right_index=True), all_branch_list)

    return return_df

In [34]:
df_door_years = door_count_df(df_door)
# df_door_years

In [35]:
df_large = pd.merge(df_door_years, df_count, left_index=True, right_index=True)

In [36]:
def produce_r(df):
    r_list = []

    branches = ['BAL', 'BEA', 'BRO', 'CAP',
        'CEN', 'COL', 'DLR', 'DTH', 'FRE', 'GLK', 'GWD', 'HIP', 'IDC', 'LCY',
        'MAG', 'MGM', 'MON', 'NET', 'NGA', 'NHY', 'QNA', 'RBE', 'SPA', 'SWT', 
        'UNI', 'WAL','WTS']

    for branch in branches:
        branch_r = df[branch].corr(df['Checkouts'])
        r_list.append(branch_r)

    return branches, r_list

In [37]:
branch_list, all_r = produce_r(df_large)

In [38]:
branch_dictionary = pd.read_csv('../ProjectData/dictionary.csv')
# branch_dictionary.head()

In [39]:
branch_names = [    ]
branch_dictionary.apply(lambda x: branch_names.append(x['Description']) if (x['Code'].upper() in branch_list) else 0, axis = 1)

0      0.0
1      0.0
2      0.0
3      NaN
4      NaN
      ... 
581    0.0
582    0.0
583    0.0
584    0.0
585    0.0
Length: 586, dtype: float64

In [40]:
plot_df = pd.DataFrame({'branch': branch_names, 'r_values': all_r})

In [41]:
chart = alt.Chart(plot_df).mark_circle().encode(
    x = alt.X('branch:N', sort='y'),
    y = alt.Y('r_values:Q', title='Relation of Visits to Checkouts'),
    color = alt.Color('branch:N'),
    tooltip = [alt.Tooltip('branch:N', title='Branch: ')]
).properties(
    title = 'Correlation of door counts to circulation 2010 to 2019',
    width = 800,
    height = 500
)

zero_mark = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(color='red').encode(
    y = 'y'
)

In [42]:
useage = chart + zero_mark
useage

alt.LayerChart(...)

In [43]:
useage.save('relation_visits_checkouts.html')

In [44]:
branch_dictionary['upper'] = branch_dictionary.apply(lambda x: x['Code'].upper(), axis=1)
bd = branch_dictionary.copy()

In [45]:
def key_branches(df):
    branch_dict = {}

    branches = ['BAL', 'BEA', 'BRO', 'CAP',
        'CEN', 'COL', 'DLR', 'DTH', 'FRE', 'GLK', 'GWD', 'HIP', 'IDC', 'LCY',
        'MAG', 'MGM', 'MON', 'NET', 'NGA', 'NHY', 'QNA', 'RBE', 'SPA', 'SWT', 
        'UNI', 'WAL','WTS']

    for branch in branches:
        rw = df[df['upper'] == branch]
        name = list(rw['Description'])[0]
        branch_dict[branch] = name

    return branch_dict

In [46]:
df_p = key_branches(bd)

In [47]:
df_plot = df_door_years.rename(df_p, axis=1)
df_plot = df_plot.reset_index()
df_plot = pd.melt(df_plot, id_vars='year', value_vars=branch_names)
# df_plot

In [48]:
branch_traffic = alt.Chart(df_plot).mark_line().encode(
    x = alt.X('year:N'),
    y = alt.Y('value:Q'),
    color = alt.Color('variable:N'),
    tooltip = [alt.Tooltip('variable:N', title='Branch: ')]
).properties(
    title = 'Branch Door counts by year',
    width = 650,
    height = 700
)

In [49]:
branch_traffic

alt.Chart(...)

In [50]:
branch_traffic.save('branch_traffic.html')

In [51]:
df_plot2 = df_plot[df_plot['variable'] != 'Central Library']

In [52]:
branch_traffic2 = alt.Chart(df_plot2).mark_line().encode(
    x = alt.X('year:N'),
    y = alt.Y('value:Q'),
    color = alt.Color('variable:N'),
    tooltip = [alt.Tooltip('variable:N', title='Branch: ')]
).properties(
    title = 'Branch Door counts by year',
    width = 650,
    height = 700
)

In [53]:
branch_traffic2

alt.Chart(...)

In [54]:
branch_traffic2.save('branch_traffice_wo_cental.html')